# Predicting babyweight using BigQuery ML

This notebook illustrates:
<ol>
<li> Machine Learning using BigQuery
<li> Make a Prediction with BQML using the Model
</ol>

Please see [this notebook](1_explore.ipynb) for more context on this problem and how the features were chosen.

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
# Ensure the right version of Tensorflow is installed.
!pip freeze | grep tensorflow==2.1

In [3]:
# change these to try this notebook out
PROJECT = 'qwiklabs-gcp-01-8bf222d23d42'
REGION = 'us-central1'

In [4]:
import os
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [5]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Exploring the Data

Here, we will be taking natality data and training on features to predict the birth weight.

The CDC's Natality data has details on US births from 1969 to 2008 and is available in BigQuery as a public data set. More details: https://bigquery.cloud.google.com/table/publicdata:samples.natality?tab=details

Lets start by looking at the data since 2000 with useful values > 0!

In [6]:
%%bigquery
SELECT
    *
FROM
  publicdata.samples.natality
WHERE
  year > 2000
  AND gestation_weeks > 0
  AND mother_age > 0
  AND plurality > 0
  AND weight_pounds > 0
LIMIT 10

,source_year,year,month,day,wday,state,is_male,child_race,weight_pounds,plurality,...,alcohol_use,drinks_per_week,weight_gain_pounds,born_alive_alive,born_alive_dead,born_dead,ever_born,father_race,father_age,record_weight
0,2001,2001,10,NaN,4,CA,False,9,6.999677,1,...,None,NaN,99,1,0,0,2,1,39,1
1,2001,2001,5,NaN,7,HI,False,9,6.768191,1,...,False,NaN,51,0,0,2,1,6,20,1
2,2001,2001,10,NaN,2,KS,False,9,7.625790,1,...,False,NaN,15,3,0,0,4,1,36,1
3,2001,2001,1,NaN,4,NC,True,9,6.563162,1,...,False,NaN,21,0,0,0,1,1,33,1
4,2001,2001,9,NaN,3,TX,False,9,6.563162,1,...,False,NaN,30,1,0,0,2,1,31,1
5,2001,2001,1,NaN,5,WA,True,9,3.844862,1,...,False,NaN,1,1,0,0,2,9,99,1
6,2001,2001,8,NaN,5,WA,False,9,8.564959,1,...,None,NaN,99,1,0,0,2,1,31,1
7,2001,2001,7,NaN,4,FL,False,9,8.313632,1,...,False,NaN,22,3,0,0,4,1,39,1
8,2001,2001,8,NaN,3,FL,True,9,6.188376,1,...,False,NaN,18,0,0,0,1,1,30,1
9,2001,2001,8,NaN,2,FL,False,9,5.187477,1,...,False,NaN,36,1,0,0,2,9,99,1


## Define Features

Looking over the data set, there are a few columns of interest that could be leveraged into features for a reasonable prediction of approximate birth weight.

Further, some feature engineering may be accomplished with the BigQuery `CAST` function -- in BQML, all strings are considered categorical features and all numeric types are considered continuous ones.

The hashmonth is added so that we can repeatably split the data without leakage -- we want all babies that share a birthday to be either in training set or in test set and not spread between them (otherwise, there would be information leakage when it comes to triplets, etc.)

In [7]:
%%bigquery
SELECT
    weight_pounds, -- this is the label; because it is continuous, we need to use regression
    CAST(is_male AS STRING) AS is_male,
    mother_age,
    CAST(plurality AS STRING) AS plurality,
    gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE
  year > 2000
  AND gestation_weeks > 0
  AND mother_age > 0
  AND plurality > 0
  AND weight_pounds > 0
LIMIT 10

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.686620,true,18,1,43,8904940584331855459
1,9.360828,true,32,1,41,1088037545023002395
2,8.437091,false,30,1,39,5896567601480310696
3,6.124442,false,24,1,40,-6244544205302024223
4,7.125340,false,26,1,41,-8029892925374153452
5,6.944561,false,31,1,40,-2126480030009879160
6,6.999677,true,23,1,40,1088037545023002395
7,9.374055,true,34,1,40,-1525201076796226340
8,8.373157,true,33,1,40,3408502330831153141
9,7.187070,false,23,1,40,-1403073183891835564


## Train Model

With the relevant columns chosen to accomplish predictions, it is then possible to create (train) the model in BigQuery. First, a dataset will be needed store the model. (if this throws an error in Datalab, simply create the dataset from the BigQuery console).

In [8]:
%%bash
bq --location=US mk -d demo

Dataset 'qwiklabs-gcp-01-8bf222d23d42:demo' successfully created.


With the demo dataset ready, it is possible to create a linear regression model to train the model.

This will take approximately **4 minutes** to run and will show **Done** when complete.

In [11]:
%%bigquery
CREATE or REPLACE MODEL demo.babyweight_model_asis
OPTIONS
  (model_type='linear_reg', labels=['weight_pounds']) AS
  
WITH natality_data AS (
  SELECT
    weight_pounds,-- this is the label; because it is continuous, we need to use regression
    CAST(is_male AS STRING) AS is_male,
    mother_age,
    CAST(plurality AS STRING) AS plurality,
    gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
)

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    natality_data
WHERE
  ABS(MOD(hashmonth, 4)) < 3  -- select 75% of the data as training

""


## Training Statistics

During the model training (and after the training), it is possible to see the model's training evaluation statistics.

For each training run, a table named `<model_name>_eval` is created. This table has basic performance statistics for each iteration.

While the new model is training, review the training statistics in the BigQuery UI to see the below model training: https://bigquery.cloud.google.com/

Since these statistics are updated after each iteration of model training, you will see different values for each refresh while the model is training.

The training details may also be viewed after the training completes from this notebook.

In [12]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL demo.babyweight_model_asis);

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,1.130197,1.129226,NaN,27213


Some of these columns are obvious although what do the non-specific ML columns mean (specific to BQML)?

**training_run** - Will be zero for a newly created model. If the model is re-trained using warm_start, this will increment for each re-training.

**iteration** - Number of the associated `training_run`, starting with zero for the first iteration.

**duration_ms** - Indicates how long the iteration took (in ms).

Note: You can also see these stats by refreshing the BigQuery UI window, finding the `<model_name>` table, selecting on it, and then the Training Stats sub-header.

Observe the training and evaluation loss and see if the model has an overfit.

## Make a Prediction with BQML using the Model

With a trained model, it is now possible to make a prediction on the values. The only difference from the second query above is the reference to the model. The data has been limited (`LIMIT 100`) to reduce amount of data returned.

When the `ml.predict` function is leveraged, output prediction column name for the model is `predicted_<label_column_name>`.

In [13]:
%%bigquery
SELECT
  *
FROM
  ml.PREDICT(MODEL demo.babyweight_model_asis,
      (SELECT
        weight_pounds,
        CAST(is_male AS STRING) AS is_male,
        mother_age,
        CAST(plurality AS STRING) AS plurality,
        gestation_weeks
      FROM
        publicdata.samples.natality
      WHERE
        year > 2000
        AND gestation_weeks > 0
        AND mother_age > 0
        AND plurality > 0
        AND weight_pounds > 0
    ))
LIMIT 100

,predicted_weight_pounds,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.032206,7.936641,false,30,1,38
1,7.654865,8.624484,true,34,1,39
2,7.415696,6.937947,true,23,1,39
3,7.306982,6.935743,true,18,1,39
4,7.402910,8.318041,false,23,1,40
...,...,...,...,...,...,...
95,7.358886,7.142977,false,33,1,39
96,7.807603,7.561856,true,29,1,40
97,7.381168,7.123136,false,22,1,40
98,6.160881,7.125340,false,26,1,35


# More advanced...

In the original example, we were taking into account the idea that if no ultrasound has been performed, some of the features (e.g. is_male) will not be known. Therefore, we augmented the dataset with such masked features and trained a single model to deal with both these scenarios.

In addition, during data exploration, we learned that the data size set for mothers older than 45 was quite sparse, so we will discretize the mother age.

In [14]:
%%bigquery
SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    CAST(plurality AS STRING) AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
LIMIT 25

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.686620,true,18,1,43,8904940584331855459
1,9.360828,true,32,1,41,1088037545023002395
2,8.437091,false,30,1,39,5896567601480310696
3,6.124442,false,24,1,40,-6244544205302024223
4,7.125340,false,26,1,41,-8029892925374153452
5,6.944561,false,31,1,40,-2126480030009879160
6,6.999677,true,23,1,40,1088037545023002395
7,9.374055,true,34,1,40,-1525201076796226340
8,8.373157,true,33,1,40,3408502330831153141
9,7.187070,false,23,1,40,-1403073183891835564


On the same dataset, will also suppose that it is unknown whether the child is male or female (on the same dataset) to simulate that an ultrasound was not been performed.

In [15]:
%%bigquery
SELECT
    weight_pounds,
    'Unknown' AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    IF(plurality > 1, 'Multiple', 'Single') AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
LIMIT 25

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.063611,Unknown,32,Single,37,7108882242435606404
1,4.687028,Unknown,30,Multiple,33,-7170969733900686954
2,7.561856,Unknown,20,Single,39,6392072535155213407
3,7.561856,Unknown,31,Single,37,-2126480030009879160
4,7.312733,Unknown,32,Single,40,3408502330831153141
5,7.627994,Unknown,30,Single,40,-2126480030009879160
6,7.251004,Unknown,33,Single,37,3408502330831153141
7,7.500126,Unknown,23,Single,39,5896567601480310696
8,7.125340,Unknown,33,Single,39,-7146494315947640619
9,7.749249,Unknown,31,Single,39,-7146494315947640619


Bringing these two separate data sets together, there is now a dataset for male or female children determined with ultrasound or unknown if without. 

In [16]:
%%bigquery
WITH with_ultrasound AS (
  SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    CAST(plurality AS STRING) AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
),

without_ultrasound AS (
  SELECT
    weight_pounds,
    'Unknown' AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    IF(plurality > 1, 'Multiple', 'Single') AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
),

preprocessed AS (
  SELECT * from with_ultrasound
  UNION ALL
  SELECT * from without_ultrasound
)

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    preprocessed
WHERE
  ABS(MOD(hashmonth, 4)) < 3
LIMIT 25

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.568469,Unknown,22,Single,46
1,8.313632,Unknown,23,Single,35
2,8.000575,Unknown,27,Single,40
3,6.563162,Unknown,29,Single,39
4,7.125340,Unknown,34,Single,40
5,8.062305,Unknown,LOW,Single,40
6,7.251004,Unknown,LOW,Single,39
7,7.561856,Unknown,27,Single,39
8,4.468770,Unknown,24,Single,33
9,6.007597,Unknown,27,Single,39


## Create a new model

With a data set which has been feature engineered, it is ready to create model with the `CREATE or REPLACE MODEL` statement

This will take **5-10 minutes** and will show **Done** when complete.

In [17]:
%%bigquery
CREATE or REPLACE MODEL demo.babyweight_model_fc
OPTIONS
  (model_type='linear_reg', labels=['weight_pounds']) AS
  
WITH with_ultrasound AS (
  SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    CAST(plurality AS STRING) AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
),

without_ultrasound AS (
  SELECT
    weight_pounds,
    'Unknown' AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    IF(plurality > 1, 'Multiple', 'Single') AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
),

preprocessed AS (
  SELECT * from with_ultrasound
  UNION ALL
  SELECT * from without_ultrasound
)

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    preprocessed
WHERE
  ABS(MOD(hashmonth, 4)) < 3

""


## Training Statistics

While the new model is training, review the training statistics in the BigQuery UI to see the below model training: https://bigquery.cloud.google.com/

The training details may also be viewed after the training completes from this notebook.

In [18]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL demo.babyweight_model_fc);

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,1.062619,1.061496,NaN,32564


## Make a prediction with the new model



Perhaps it is of interest to make a prediction of the baby's weight given a number of other factors: Male, Mother is 28 years old, Mother will only have one child, and the baby was born after 38 weeks of pregnancy.

To make this prediction, these values will be passed into the SELECT statement.

In [19]:
%%bigquery
SELECT
  *
FROM
  ml.PREDICT(MODEL demo.babyweight_model_fc,
      (SELECT
          'True' AS is_male,
          '28' AS mother_age,
          '1' AS plurality,
          '38' AS gestation_weeks
    ))

,predicted_weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,-31162.451016,True,28,1,38


<br>
<br>
<br>
<br>
Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License